# Carregando os dados do DuckDB

O objetivo dessa etapa é realizar a carga nos dados no banco duckdb.

## Etapas realizadas neste código:
1. Importação das bibliotecas necessárias (duckdb, pandas, time, os, datetime).
2. Definição de uma função de log para registrar mensagens com timestamp.
3. Mapeamento dos arquivos de uma pasta para um dicionário com nomes e caminhos completos.
4. Conexão ao banco de dados DuckDB.
5. Criação/verificação do schema 'raw' no banco de dados.
6. Inserção dos arquivos mapeados como tabelas no schema 'raw' do DuckDB, utilizando detecção automática de formato.
7. Criação das demais zonas de dados


## 1- Import e criação das funções

In [ ]:
import duckdb
import pandas as pd
import time
import os
from datetime import datetime

# Função utilitária para log com horário
def log(msg):
    print(f"[{datetime.now().strftime('%Y-%m-%d %H:%M:%S')}] {msg}")

In [ ]:
def mapear_arquivos_pasta(pasta):
    """
    Mapeia todos os arquivos da pasta e retorna um dicionário
    com o nome do arquivo como chave e o caminho completo como valor.
    """
    arquivos_dict = {}
    for root, dirs, files in os.walk(pasta):
        for file in files:
            caminho_completo = os.path.join(root, file)
            arquivos_dict[file] = caminho_completo
    return arquivos_dict

In [ ]:
def inserir_arquivos_no_duckdb(arquivos_dict, con):
    """
    Para cada arquivo no dicionário, checa se a tabela existe.
    Se não existir, insere usando fast-arrow (COPY ... FROM ... AUTO_DETECT).
    """
    
    # Garante que o schema 'raw' existe antes de inserir os dados
    con.execute("CREATE SCHEMA IF NOT EXISTS raw;")
    print("📁 Schema criado/verificado: raw")
    # Adiciona os dados no schema 'raw'
    for nome_arquivo, caminho_arquivo in arquivos_dict.items():
    # Garante que o schema 'raw' existe antes de inserir os dados
        try:
            con.execute("CREATE SCHEMA IF NOT EXISTS raw;")
            print("📁 Schema criado/verificado: raw")
        except Exception as e:
            log(f"Erro ao criar/verificar schema 'raw': {e}")
            raise
        # Adiciona os dados no schema 'raw'
        for nome_arquivo, caminho_arquivo in arquivos_dict.items():
            nome_tabela = os.path.splitext(nome_arquivo)[0]
            tabela_com_schema = f"raw.{nome_tabela}"
            # Checa se a tabela já existe no schema 'raw'
            existe = con.execute(
                f"SELECT COUNT(*) FROM information_schema.tables WHERE table_schema = 'raw' AND table_name = '{nome_tabela}'"
            ).fetchone()[0]
            if existe:
                log(f"Tabela '{tabela_com_schema}' já existe. Pulando inserção.")
                continue
            # Detecta tipo de arquivo e monta comando COPY
            if caminho_arquivo.endswith(".parquet"):
                comando = f"""
                    CREATE TABLE {tabela_com_schema} AS SELECT * FROM read_parquet('{caminho_arquivo}')
                """
            elif caminho_arquivo.endswith(".csv") or caminho_arquivo.endswith(".CSV"):
                comando = f"""
                    CREATE TABLE {tabela_com_schema} AS SELECT * FROM read_csv_auto('{caminho_arquivo}')
                """
            else:
                log(f"Formato de arquivo não suportado para '{caminho_arquivo}'. Pulando.")
                continue
            log(f"Inserindo '{caminho_arquivo}' na tabela '{tabela_com_schema}'...")
            con.execute(comando)
            log(f"Tabela '{tabela_com_schema}' criada e dados inseridos com sucesso.")

In [ ]:
def listar_tabelas(conexao):
    """
    Retorna uma lista com o nome de todas as tabelas existentes no banco DuckDB conectado.
    """
    resultado = conexao.execute(
        "SELECT table_name FROM information_schema.tables WHERE table_schema = 'raw'"
    ).fetchall()
    return [linha[0] for linha in resultado]

## 2: Definição de entradas

In [ ]:
# Caminho do arquivo (pode ser CSV, Parquet, etc)
PATH_DIRETORIO = "data_input"

# Caminho do banco DuckDB (arquivo .db)
CAMINHO_DUCKDB = "bd/dev.duckdb"

# Cria a conexão com o banco DuckDB
con = duckdb.connect(CAMINHO_DUCKDB)


In [ ]:
inserir_arquivos_no_duckdb(mapear_arquivos_pasta(PATH_DIRETORIO), con)


📁 Schema criado/verificado: raw
📁 Schema criado/verificado: raw
[2025-09-02 10:54:36] Inserindo 'data_input\gestor_2019.CSV' na tabela 'raw.gestor_2019'...
[2025-09-02 10:54:36] Tabela 'raw.gestor_2019' criada e dados inseridos com sucesso.
[2025-09-02 10:54:36] Inserindo 'data_input\gestor_2020.CSV' na tabela 'raw.gestor_2020'...
[2025-09-02 10:54:37] Tabela 'raw.gestor_2020' criada e dados inseridos com sucesso.
📁 Schema criado/verificado: raw
[2025-09-02 10:54:37] Tabela 'raw.gestor_2019' já existe. Pulando inserção.
[2025-09-02 10:54:37] Tabela 'raw.gestor_2020' já existe. Pulando inserção.


In [ ]:
listar_tabelas(con)

['gestor_2019', 'gestor_2020']

In [ ]:
#Criação dos schemas
for schema in  ["staging", "intermediate", "mart"]:
    con.execute(f"CREATE SCHEMA IF NOT EXISTS {schema};")


In [ ]:
con.close()